In [7]:
# importing required classes
from pypdf import PdfReader, PdfWriter

In [8]:
import xlsxwriter, pandas as pd

In [9]:
composicoes = {
    "servico_title": [
        "servico",
        "descricao",
        "producao",
        "unidade",
        "custo unitario execucao",
        "custo direto",
        "bdi",
        "preco unitario total",
        "pagina",
    ],
    "equipamentos_title": [
        "servico",
        "equipamento",
        "codigo",
        "ut. pr.",
        "ut. impr.",
        "vl. hr. prod.",
        "vl. hr. impr.",
        "consumo",
        "custo horario",
    ],
    "mao_de_obra_title": [
        "servico",
        "mao de obra",
        "codigo",
        "eq. salarial",
        "sal. hora",
        "% encargos",
        "consumo",
        "custo horario",
    ],
    "itens_de_incidencia_title": [
        "servico",
        "itens de incidencia",
        "codigo",
        "%",
        "M.O.",
        # "equip.",
        # "mat.",
        "custo",
    ],
    "materiais_title": [
        "servico",
        "materiais",
        "codigo",
        "unidade",
        "custo unitario",
        "consumo",
        "custo",
    ],
    "servicos_title": [
        "servico",
        "servicos",
        "codigo",
        "unidade",
        "custo unitario",
        "consumo",
        "custo",
    ],
    "itens_de_transporte_title": [
        "servico",
        "itens de transporte",
        "codigo",
        "unidade",
        "formula",
        "x1",
        "x2",
        "x3",
        "custo unitario",
        "consumo",
        "custo",
    ],
}

In [10]:
def add_sheet(wb, sheet_name, sheet_title):
    ws = wb.add_worksheet(sheet_name)
    for n, i in enumerate(sheet_title):
        ws.write(0, n, i)

In [11]:
def add_data_sheet(wb, sheet_name, sheet_vars):
    row = 1
    ws = wb.get_worksheet_by_name(sheet_name)
    for i in sheet_vars:
        for n, v in enumerate(i):
            ws.write(row, n, v)
        row += 1

In [12]:
number_format = {",": "", "": ""}
number_format_br = {".": "", ",": "."}

In [13]:
composicoes_files = [
    "source/composicoes.pdf",
    "source/composicoes---t109.pdf",
]

global_composicoes = {}
_global_servico = []
_global_equipamentos = []
_global_mao_de_obra = []
_global_itens_de_incidencia = []
_global_producao_equipe = []
_global_materiais = []
_global_servicos = []
_global_itens_de_transporte = []
for composicoes_file in composicoes_files:  # creating a pdf reader object
    global_servico = []
    global_equipamentos = []
    global_mao_de_obra = []
    global_itens_de_incidencia = []
    global_producao_equipe = []
    global_materiais = []
    global_servicos = []
    global_itens_de_transporte = []
    global_pages = {}

    target = composicoes_file.replace("source", "target").replace(".pdf", ".xlsx")
    print("target:", target)
    wb = xlsxwriter.Workbook(target)
    for k, v in composicoes.items():
        sheet = k.replace("_title", "").replace("_", " ")
        # print(sheet, v)
        add_sheet(wb, sheet, v)

    reader = PdfReader(composicoes_file)
    # print((reader.pages))

    for n, page in enumerate(reader.pages):
        # extracting text from page
        texts = page.extract_text().splitlines()

        is_a = False
        __t = ""
        is_b = False
        __t = ""
        have_encargos = False
        is_c = False
        is_not_c = False
        is_d = False
        is_e = False
        is_f = False
        is_g = False
        is_h = False
        is_not_h = False
        equipamento_s = []
        mao_de_obra_s = []
        itens_de_incidencia_s = []
        producao_equipe_s = []
        materiais_s = []
        servicos_s = []
        itens_de_transporte_s = []
        for text in texts:
            text_left = text.lstrip().rstrip()
            if text_left.startswith("Serviço"):
                servico = text_left[
                    text_left.find("Serviço")
                    + len("Serviço")
                    + 2 : text_left.find("Serviço")
                    + len("Serviço")
                    + 7
                ].strip()
                servico_nome = text_left[
                    text_left.find("Serviço")
                    + len("Serviço")
                    + 9 : text_left.find("Unidade")
                    - 1
                ].strip()
                # print("servico:", servico)
                servico_page = {servico: n}
                global_pages.update(servico_page)
                _composicoes = {composicoes_file: global_pages}
                global_composicoes.update(_composicoes)
            if "Unidade: " in text_left:
                unidade = text_left[
                    text_left.find("Unidade") + len("Unidade") + 2 :
                ].strip()
                # print(servico, unidade)
            if text_left.startswith("(A)"):
                is_a = True
            if text_left.startswith("(B)"):
                is_b = True
            if "Encargos(%)" in text_left:
                have_encargos = True
            if text_left.startswith("(C)"):
                is_c = True
            if text_left.startswith("Custo Horário"):
                is_not_c = True
            if text_left.startswith("(D)"):
                is_d = True
            if text_left.startswith("(E)"):
                is_e = True
            if text_left.startswith("(F)"):
                is_f = True
            if text_left.startswith("(G)"):
                is_g = True
                is_f = False
            if text_left.startswith("(H)"):
                is_h = True
                is_g = False
            if text_left.startswith("(H)Total"):
                is_h = False
            if text_left.startswith("Custo Direto"):
                is_h = False
            if not (text_left.startswith("auxiliar")):
                if is_a == True and is_b == False:
                    if not text_left.startswith("(A)"):
                        _equipamento = text.split()
                        if (
                            "," not in _equipamento[-1:][0]
                            and "." not in _equipamento[-1:][0]
                        ):
                            __t = text_left
                        else:
                            equipamento = [
                                " ".join(
                                    filter(
                                        None,
                                        [__t]
                                        + _equipamento[
                                            : len(_equipamento)
                                            - (7 if __t == "" else 6)
                                        ],
                                    )
                                )
                            ] + (
                                [
                                    "".join(
                                        [
                                            number_format.get(char, char)
                                            for char in fields
                                        ]
                                    )
                                    for fields in _equipamento[-7:]
                                ]
                                if composicoes_file == "source/composicoes.pdf"
                                else _equipamento[-7:]
                            )

                            if __t != "":
                                equipamento[0] = equipamento[0][:-5]
                                equipamento[1] = equipamento[1][-5:]
                            equipamento_s.append(equipamento)
                            __t = ""
                if is_b == True and is_c == False:
                    if not text_left.startswith("(B)"):
                        _mao_de_obra = text.split()
                        # print(_mao_de_obra)
                        if (
                            "," not in _mao_de_obra[-1:][0]
                            and "." not in _mao_de_obra[-1:][0]
                        ):
                            __t = text_left
                        else:
                            mao_de_obra = [
                                " ".join(
                                    filter(
                                        None,
                                        [__t]
                                        + _mao_de_obra[
                                            : len(_mao_de_obra)
                                            - (5 if __t == "" else 4)
                                            - (1 if have_encargos == True else 0)
                                        ],
                                    )
                                )
                            ] + (
                                [
                                    "".join(
                                        [
                                            number_format.get(char, char)
                                            for char in fields
                                        ]
                                    )
                                    for fields in _mao_de_obra[
                                        -5 - (1 if have_encargos == True else 0) :
                                    ]
                                ]
                                if composicoes_file == "source/composicoes.pdf"
                                else _mao_de_obra[
                                    -5 - (1 if have_encargos == True else 0) :
                                ]
                            )
                            if have_encargos == False:
                                mao_de_obra.insert(4, "-")
                            if __t != "":
                                mao_de_obra[0] = mao_de_obra[0][:-5]
                                mao_de_obra[1] = mao_de_obra[1][-5:]
                            mao_de_obra_s.append(mao_de_obra)
                            __t = ""
                if is_c == True and is_not_c == False:
                    if not text_left.startswith("(C)") and not text_left.startswith(
                        "Custo Horário"
                    ):
                        _itens_de_incidencia = text.split()
                        if (
                            "," not in _itens_de_incidencia[-1:][0]
                            and "." not in _itens_de_incidencia[-1:][0]
                        ):
                            __t = text_left
                        else:
                            itens_de_incidencia = [
                                " ".join(
                                    filter(
                                        None,
                                        [__t]
                                        + _itens_de_incidencia[
                                            : len(_itens_de_incidencia)
                                            - (4 if __t == "" else 3)
                                        ],
                                    )
                                )
                            ] + (
                                [
                                    "".join(
                                        [
                                            number_format.get(char, char)
                                            for char in fields
                                        ]
                                    )
                                    for fields in _itens_de_incidencia[-4:]
                                ]
                                if composicoes_file == "source/composicoes.pdf"
                                else _itens_de_incidencia[-4:]
                            )
                            if __t != "":
                                itens_de_incidencia[0] = itens_de_incidencia[0][:-5]
                                itens_de_incidencia[1] = itens_de_incidencia[1][-5:]
                            itens_de_incidencia_s.append(itens_de_incidencia)
                            __t = ""
                if is_f == True and is_g == False:
                    if not text_left.startswith("(F)"):
                        _materiais = text.split()
                        if (
                            "," not in _materiais[-1:][0]
                            and "." not in _materiais[-1:][0]
                        ):
                            __t = text_left
                        else:
                            materiais = [
                                " ".join(
                                    filter(
                                        None,
                                        [__t]
                                        + _materiais[
                                            : len(_materiais) - (5 if __t == "" else 4)
                                        ],
                                    )
                                )
                            ] + (
                                [
                                    "".join(
                                        [
                                            number_format.get(char, char)
                                            for char in fields
                                        ]
                                    )
                                    for fields in _materiais[-5:]
                                ]
                                if composicoes_file == "source/composicoes.pdf"
                                else _materiais[-5:]
                            )
                            if __t != "":
                                materiais[0] = materiais[0][:-5]
                                materiais[1] = materiais[1][-5:]
                            materiais_s.append(materiais)
                            __t = ""
                if is_g == True and is_h == False:
                    if not text_left.startswith("(G)"):
                        _servicos = text.split()
                        if (
                            "," not in _servicos[-1:][0]
                            and "." not in _servicos[-1:][0]
                        ):
                            __t = text_left
                        else:
                            servicos = [
                                " ".join(
                                    filter(
                                        None,
                                        [__t]
                                        + _servicos[
                                            : len(_servicos) - (5 if __t == "" else 4)
                                        ],
                                    )
                                )
                            ] + (
                                [
                                    "".join(
                                        [
                                            number_format.get(char, char)
                                            for char in fields
                                        ]
                                    )
                                    for fields in _servicos[-5:]
                                ]
                                if composicoes_file == "source/composicoes.pdf"
                                else _servicos[-5:]
                            )
                            if __t != "":
                                servicos[0] = servicos[0][:-5]
                                servicos[1] = servicos[1][-5:]
                            servicos_s.append(servicos)
                            __t = ""
                if is_h == True:
                    if not text_left.startswith("(H)"):
                        _itens_de_transporte = text.split()
                        if (
                            "," not in _itens_de_transporte[-1:][0]
                            and "." not in _itens_de_transporte[-1:][0]
                        ):
                            __t = text_left
                        else:
                            itens_de_transporte = [
                                " ".join(
                                    filter(
                                        None,
                                        [__t]
                                        + _itens_de_transporte[
                                            : len(_itens_de_transporte)
                                            - (9 if __t == "" else 8)
                                        ],
                                    )
                                )
                            ] + (
                                [
                                    "".join(
                                        [
                                            number_format.get(char, char)
                                            for char in fields
                                        ]
                                    )
                                    for fields in _itens_de_transporte[-9:]
                                ]
                                if composicoes_file == "source/composicoes.pdf"
                                else _itens_de_transporte[-9:]
                            )

                            if __t != "":
                                itens_de_transporte[0] = itens_de_transporte[0][:-5]
                                itens_de_transporte[1] = itens_de_transporte[1][-5:]
                            itens_de_transporte_s.append(itens_de_transporte)
                            __t = ""
                if text_left.startswith("(D) Produção da Equipe"):
                    producao = text_left[
                        text_left.find("(D) Produção da Equipe")
                        + len("(D) Produção da Equipe")
                        + 1 :
                    ].strip()
                    print("producao:", producao)
                    producao = (
                        "".join([number_format.get(char, char) for char in producao])
                        if composicoes_file == "source/composicoes.pdf"
                        else float(
                            "".join(
                                [number_format_br.get(char, char) for char in producao]
                            )
                        )
                    )
                    print("producao:", producao)
                if text_left.startswith("(E)"):
                    custo_unitario_execucao = (
                        text_left[text_left.find("(E)") + len("(E)") + 1 :]
                        .strip()
                        .split()[-1]
                    )
                    custo_unitario_execucao = (
                        "".join(
                            [
                                number_format.get(char, char)
                                for char in custo_unitario_execucao
                            ]
                        )
                        if composicoes_file == "source/composicoes.pdf"
                        else custo_unitario_execucao
                    )
                if text_left.startswith("Custo Direto"):
                    custo_direto = (
                        text_left[
                            text_left.find("Custo Direto") + len("Custo Direto") + 1 :
                        ]
                        .strip()
                        .split()[-1]
                    )
                    custo_direto = (
                        "".join(
                            [number_format.get(char, char) for char in custo_direto]
                        )
                        if composicoes_file == "source/composicoes.pdf"
                        else custo_direto
                    )
                if text_left.startswith("BDI"):
                    bdi = (
                        text_left[text_left.find("BDI") + len("BDI") + 1 :]
                        .strip()
                        .split()[-1]
                    )
                    bdi = (
                        "".join([number_format.get(char, char) for char in bdi])
                        if composicoes_file == "source/composicoes.pdf"
                        else bdi
                    )
                if text_left.startswith("Preço Unitário"):
                    preco_unitario_total = (
                        text_left[
                            text_left.find("Preço Unitário")
                            + len("Preço Unitário")
                            + 1 :
                        ]
                        .strip()
                        .split()[-1]
                    )
                    preco_unitario_total = (
                        "".join(
                            [
                                number_format.get(char, char)
                                for char in preco_unitario_total
                            ]
                        )
                        if composicoes_file == "source/composicoes.pdf"
                        else preco_unitario_total
                    )
                    # unidade = text_left[text_left.find("Unidade") + len("Unidade") + 2 :]
                    servicos = [
                        servico,
                        servico_nome,
                        producao,
                        unidade,
                        custo_unitario_execucao,
                        custo_direto,
                        bdi,
                        preco_unitario_total,
                        n + 1,
                    ]

        global_servico.append(servicos)
        _global_servico.append(servicos + [composicoes_file])
        for i in equipamento_s:
            i = [servico] + i
            global_equipamentos.append(i)
            _global_equipamentos.append(i + [composicoes_file])
        for i in mao_de_obra_s:
            i = [servico] + i
            global_mao_de_obra.append(i)
            _global_mao_de_obra.append(i + [composicoes_file])
        for i in itens_de_incidencia_s:
            i = [servico] + i
            global_itens_de_incidencia.append(i)
            _global_itens_de_incidencia.append(i + [composicoes_file])
        for i in producao_equipe_s:
            i = [servico] + i
            global_producao_equipe.append(i)
            _global_producao_equipe.append(i + [composicoes_file])
        for i in materiais_s:
            i = [servico] + i
            global_materiais.append(i)
            _global_materiais.append(i + [composicoes_file])
        for i in servicos_s:
            i = [servico] + i
            global_servicos.append(i)
            _global_servicos.append(i + [composicoes_file])
        for i in itens_de_transporte_s:
            i = [servico] + i
            global_itens_de_transporte.append(i)
            _global_itens_de_transporte.append(i + [composicoes_file])

    for k, v in composicoes.items():
        add_data_sheet(
            wb,
            k.replace("_title", "").replace("_", " "),
            globals()[f"global_{k.replace('_title', '')}"],
        )
    wb.close()

target: target/composicoes.xlsx
producao: 1,144.00
producao: 1144.00
producao: 1,394.00
producao: 1394.00
producao: 191.00
producao: 191.00
producao: 133.50
producao: 133.50
producao: 220.00
producao: 220.00
producao: 200.00
producao: 200.00
producao: 40.00
producao: 40.00
producao: 23.00
producao: 23.00
producao: 78.00
producao: 78.00
producao: 90.00
producao: 90.00
producao: 133.50
producao: 133.50
producao: 18.00
producao: 18.00
producao: 18.00
producao: 18.00
producao: 241.00
producao: 241.00
producao: 135.00
producao: 135.00
producao: 3.00
producao: 3.00
producao: 940.00
producao: 940.00
producao: 844.00
producao: 844.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 1.00
producao: 751.00
producao: 751.00
producao: 870.00
producao: 870.00
producao: 341.00
producao: 341.00
producao: 165.00
produca

In [14]:
_global_servicos

[['41200',
  'Escav. mec. de valas de 1ª cat. (incl. transporte)',
  '47023',
  'm3',
  '7.90',
  '0.7500',
  '5.92',
  'source/composicoes.pdf'],
 ['41205',
  'Escavação mec. de valas de mat. 3ª cat.',
  '47022',
  'm3',
  '77.81',
  '0.2500',
  '19.45',
  'source/composicoes.pdf'],
 ['41210',
  'Escavação e carga de material de 1ª cat',
  '40025',
  'm3',
  '3.66',
  '0.1000',
  '0.36',
  'source/composicoes.pdf'],
 ['41210',
  'Forma de placa compensada',
  '47020',
  'm2',
  '37.86',
  '0.5600',
  '21.20',
  'source/composicoes.pdf'],
 ['41210',
  'Concreto fck=11 MPa',
  '47025',
  'm3',
  '226.32',
  '0.2040',
  '46.16',
  'source/composicoes.pdf'],
 ['41225',
  'Confecção de guias',
  '47021',
  'M',
  '2.44',
  '0.0340',
  '0.08',
  'source/composicoes.pdf'],
 ['41225',
  'Concreto fck=11 MPa',
  '47025',
  'm3',
  '226.32',
  '0.0640',
  '14.48',
  'source/composicoes.pdf'],
 ['41225',
  'Caiação',
  '47026',
  'm2',
  '1.69',
  '0.7200',
  '1.21',
  'source/composicoes.pdf'],

In [15]:
sheets_dict = pd.read_excel("source/edital_pav_rod_sem_mob.xlsx", sheet_name=None)

all_sheets = []
for name, sheet in sheets_dict.items():
    sheet["sheet"] = name
    sheet = sheet.rename(columns=lambda x: x.split("\n")[-1])
    all_sheets.append(sheet)

full_table = pd.concat(all_sheets)
full_table.reset_index(inplace=True, drop=True)

In [16]:
full_table.rename(
    columns={
        "PROJETO BÁSICO DE ENGENHARIA": "servico",
        "Unnamed: 1": "descricao",
        "Unnamed: 2": "unidade",
        "Unnamed: 3": "quantidade",
        "Unnamed: 4": "preco unitario",
        "Unnamed: 5": "preco total",
        "sheet": "projeto",
    },
    inplace=True,
)

In [17]:
full_table["servico"] = full_table["servico"].astype(str).str.strip()

In [18]:
df = full_table[
    (~full_table["servico"].str.contains("TOTAL"))
    & (full_table["servico"].str.len() == 5)
]

In [19]:
df

,servico,descricao,unidade,quantidade,preco unitario,preco total,projeto
4,40005,"Desmatamento, destocamento e limpeza (arv. até...",m²,350400,0.19,66576,NS01
5,40015,Carga de entulhos - material de limpeza,m³,91104,1.19,108413.76,NS01
6,40020,Transporte de entulhos - material de limpeza (...,m³.km,455520,1.01,460075.2,NS01
7,40025,Escavação e carga de material de 1ª categoria,m³,557909.7758,4.68,2611017.75,NS01
8,40050,Escavação e carga de solo mole,m³,9647.2,8.11,78238.79,NS01
...,...,...,...,...,...,...,...
1053,42805,"Gabiões (0,50 m)",m³,81.04,416.7,33769.36,Vila Maria
1054,42800,"Gabiões (1,00 m)",m³,214.14,338.04,72387.88,Vila Maria
1055,42830,Concreto fck=11MPa,m³,7.86,290.49,2283.25,Vila Maria
1056,45550,Meio Fio sem sarjeta (GAP),m,14640,14.86,217550.4,Vila Maria


In [20]:
for source in composicoes_files:
    print(source)

source/composicoes.pdf
source/composicoes---t109.pdf


In [21]:
reader_composicoes = PdfReader("source/composicoes.pdf")
reader_composicoes_t109 = PdfReader("source/composicoes---t109.pdf")
projetos = {}
for projeto in df["projeto"].unique():
    writer = PdfWriter()
    servicos = df[(df["projeto"] == projeto)]["servico"].to_list()
    print(servicos)
    pages_pdf_composicoes = []
    pages_pdf_composicoes_t109 = []
    print(projeto, servicos)
    for servico in servicos:
        if servico in global_composicoes.get("source/composicoes.pdf").keys():
            pdf_page = global_composicoes.get("source/composicoes.pdf").get(
                servico, servico
            )
            pages_pdf_composicoes.append(pdf_page)
            writer.add_page(reader_composicoes.pages[pdf_page])
            print("composicoes:", pdf_page)

        elif servico in global_composicoes.get("source/composicoes---t109.pdf").keys():
            pdf_page = global_composicoes.get("source/composicoes---t109.pdf").get(
                servico, servico
            )
            pages_pdf_composicoes_t109.append(pdf_page)
            writer.add_page(reader_composicoes_t109.pages[pdf_page])
            print("composicoes---t109:", pdf_page)
        # print(global_composicoes.get("source/composicoes.pdf").get(servico, servico))
    print(pages_pdf_composicoes, pages_pdf_composicoes_t109)
    projetos.update(
        {
            projeto: {
                "composicoes": pages_pdf_composicoes,
                "composicoes---t109": pages_pdf_composicoes_t109,
            }
        }
    )
    writer.write(f"target/projetos/{projeto}.pdf")
print(projetos)

['40005', '40015', '40020', '40025', '40050', '40055', '40065', '40075', '40080', '40085', '40095', '40300', '40305', '40310', '40335', '40316', '40320', '40350', '40316', '40320', '40380', '40385', '40405', '40435', '40450', '40460', '41290', '47023', '45430', '45435', '45445', '45450', '45455', '45460', '45465', '45490', '45500', '45510', '45520', '45495', '45505', '45515', '45525', '45530', '45535', '45585', '45580', '45590', '45605', '40055', '42810', '42805', '42800', '45585', '42830', '45555', '45550', '41255', '41200', '41210', '41290', '41270', '40805', '40885', '40310', '42820', '40425', '40015', '40020', '40100', '40810', '40820', '40830', '40845', '40855', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', 

In [22]:
sheets = df["projeto"].unique()

In [23]:
sheets

array(['NS01', 'NS03', 'NS05', 'Jardim Bonança', 'Parque Santa Cecília',
       'Vila Maria'], dtype=object)

In [24]:
df

,servico,descricao,unidade,quantidade,preco unitario,preco total,projeto
4,40005,"Desmatamento, destocamento e limpeza (arv. até...",m²,350400,0.19,66576,NS01
5,40015,Carga de entulhos - material de limpeza,m³,91104,1.19,108413.76,NS01
6,40020,Transporte de entulhos - material de limpeza (...,m³.km,455520,1.01,460075.2,NS01
7,40025,Escavação e carga de material de 1ª categoria,m³,557909.7758,4.68,2611017.75,NS01
8,40050,Escavação e carga de solo mole,m³,9647.2,8.11,78238.79,NS01
...,...,...,...,...,...,...,...
1053,42805,"Gabiões (0,50 m)",m³,81.04,416.7,33769.36,Vila Maria
1054,42800,"Gabiões (1,00 m)",m³,214.14,338.04,72387.88,Vila Maria
1055,42830,Concreto fck=11MPa,m³,7.86,290.49,2283.25,Vila Maria
1056,45550,Meio Fio sem sarjeta (GAP),m,14640,14.86,217550.4,Vila Maria


In [25]:
# df_servico = pd.DataFrame(
#     _global_servico, columns=composicoes["servico_title"] + ["source"]
# )

In [26]:
composicoes["equipamentos_title"]

['servico',
 'equipamento',
 'codigo',
 'ut. pr.',
 'ut. impr.',
 'vl. hr. prod.',
 'vl. hr. impr.',
 'consumo',
 'custo horario']

In [27]:
df_servico = pd.DataFrame(
    _global_servico, columns=composicoes["servico_title"] + ["source"]
)
df_equipamentos = pd.DataFrame(
    _global_equipamentos, columns=composicoes["equipamentos_title"] + ["source"]
)
df_mao_de_obra = pd.DataFrame(
    _global_mao_de_obra, columns=composicoes["mao_de_obra_title"] + ["source"]
)
df_itens_de_incidencia = pd.DataFrame(
    _global_itens_de_incidencia,
    columns=composicoes["itens_de_incidencia_title"] + ["source"],
)
df_materiais = pd.DataFrame(
    _global_materiais, columns=composicoes["materiais_title"] + ["source"]
)
df_servicos = pd.DataFrame(
    _global_servicos, columns=composicoes["servicos_title"] + ["source"]
)
df_itens_de_transporte = pd.DataFrame(
    _global_itens_de_transporte,
    columns=composicoes["itens_de_transporte_title"] + ["source"],
)

In [28]:
df_servico
# df_equipamentos
# df_mao_de_obra
# df_itens_de_incidencia
# df_materiais
# df_servicos
# df_itens_de_transporte

,servico,descricao,producao,unidade,custo unitario execucao,custo direto,bdi,preco unitario total,pagina,source
0,40005,"Desmatamento,dest. e limpeza(arv.até 15cm)",1144.00,m2,0.15,0.15,0.04,0.19,1,source/composicoes.pdf
1,40010,"Desmatamento,dest. e limpeza(arv.D>15cm)",1394.00,m2,0.26,0.26,0.07,0.33,2,source/composicoes.pdf
2,40015,Carga de entulhos,191.00,m3,0.93,0.93,0.26,1.19,3,source/composicoes.pdf
3,40020,Transporte de entulhos,133.50,m3km,0.79,0.79,0.22,1.01,4,source/composicoes.pdf
4,40025,Escavação e carga de material de 1ª cat,220.00,m3,3.66,3.66,1.02,4.68,5,source/composicoes.pdf
...,...,...,...,...,...,...,...,...,...,...
846,45725,"FORNECIMENTO, CORTE E COLOCAÇÃO DE BAINHA METÁ...",103.0,m,"1,64","32,71","8,17","40,88",646,source/composicoes---t109.pdf
847,45730,"FORNECIMENTO E COLOCAÇÃO DE ANCORAGENS ATIVAS,...",1.0,un,"78,76","635,97","158,99","794,96",647,source/composicoes---t109.pdf
848,45735,"FORNECIMENTO E COLOCAÇÃO DE ANCORAGENS ATIVAS,...",1.0,un,"78,76","1.128,04","282,01","1.410,05",648,source/composicoes---t109.pdf
849,45740,"FORNECIMENTO E COLOCAÇÃO DE ANCORAGENS ATIVAS,...",1.0,un,"78,76","638,28","159,57","797,85",649,source/composicoes---t109.pdf


In [29]:
# # df_servico.convert_dtypes().astype({"producao": "float"}).dtypes

# # df_servico["producao"].str.replace(",", "").replace(".", ",").astype("float").dtypes

# composicoes_float = {
#     "servico": [
#         "producao",
#         "custo unitario execucao",
#         "custo direto",
#         # "bdi",
#         "preco unitario total",
#     ],
#     "equipamentos": [
#         "ut. pr.",
#         "ut. impr.",
#         "vl. hr. prod.",
#         "vl. hr. impr.",
#         "consumo",
#         "custo horario",
#     ],
#     "mao_de_obra": [
#         "eq. salarial",
#         "sal. hora",
#         "consumo",
#         "custo horario",
#     ],
#     "itens_de_incidencia": ["%", "custo"],
#     "materiais": [
#         "custo unitario",
#         "consumo",
#         "custo",
#     ],
#     "servicos": [
#         "custo unitario",
#         "consumo",
#         "custo",
#     ],
#     "itens_de_transporte": [
#         # "unidade",
#         "x1",
#         "x2",
#         # "x3",
#         "custo unitario",
#         "consumo",
#         "custo",
#     ],
# }

# for k, v in composicoes_float.items():
#     print(k)
#     # display(globals()[f"df_{k}"])
#     for _v in v:
#         print("-->", _v)
#         globals()[f"df_{k}"][f"{_v}"] = (
#             globals()[f"df_{k}"][f"{_v}"]
#             .str.replace(",", "")
#             .replace(".", ",")
#             .astype("float")
#         )

In [30]:
_df = df[df["projeto"] == "NS01"]
_df_servico = pd.merge(
    _df[
        [
            "servico",
            "descricao",
            # "unidade",
            "quantidade",
            "preco unitario",
            "preco total",
        ]
    ],
    df_servico[
        [
            "servico",
            "producao",
            "unidade",
            "custo unitario execucao",
            "custo direto",
            "bdi",
            "preco unitario total",
            "pagina",
            "source",
        ]
    ],
    # how="outer",
    on="servico",
    validate="m:m",
).drop_duplicates(subset=["servico"])
_df_servico

,servico,descricao,quantidade,preco unitario,preco total,producao,unidade,custo unitario execucao,custo direto,bdi,preco unitario total,pagina,source
0,40005,"Desmatamento, destocamento e limpeza (arv. até...",350400,0.19,66576,1144.00,m2,0.15,0.15,0.04,0.19,1,source/composicoes.pdf
2,40015,Carga de entulhos - material de limpeza,91104,1.19,108413.76,191.00,m3,0.93,0.93,0.26,1.19,3,source/composicoes.pdf
4,40020,Transporte de entulhos - material de limpeza (...,455520,1.01,460075.2,133.50,m3km,0.79,0.79,0.22,1.01,4,source/composicoes.pdf
6,40025,Escavação e carga de material de 1ª categoria,557909.7758,4.68,2611017.75,220.00,m3,3.66,3.66,1.02,4.68,5,source/composicoes.pdf
8,40050,Escavação e carga de solo mole,9647.2,8.11,78238.79,23.00,m3,6.34,6.34,1.77,8.11,8,source/composicoes.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,45135,Escoramento para ponte,41.442125,58.67,2431.4,1.0,m3,"31,34","38,73","9,68","48,41",602,source/composicoes---t109.pdf
205,45150,Forma chapa comp. Plastificada 17mm,4204.38649,81.56,342909.76,1.0,m2,"18,06","43,13","10,78","53,91",605,source/composicoes---t109.pdf
214,45215,Pavimentação-Concreto fck=20Mpa,97.347552,455.65,44356.41,1.0,m3,"178,37","372,02","93,00","465,02",632,source/composicoes---t109.pdf
215,45230,Drenos 100mm,58.018975,4.64,269.2,1.0,un,"1,26","3,79","0,94","4,73",635,source/composicoes---t109.pdf


In [31]:
for s in sheets:
    with pd.ExcelWriter(f"target/projetos/{s}.xlsx") as writer:
        _df = df[df["projeto"] == s]
        _df_servico = pd.merge(
            _df[
                [
                    "servico",
                    "descricao",
                    # "unidade",
                    "quantidade",
                    "preco unitario",
                    "preco total",
                ]
            ],
            df_servico[
                [
                    "servico",
                    "producao",
                    "unidade",
                    "custo unitario execucao",
                    "custo direto",
                    "bdi",
                    "preco unitario total",
                    "pagina",
                    "source",
                ]
            ],
            # how="outer",
            on="servico",
            validate="m:m",
        ).drop_duplicates(subset=["servico"])
        _df_servico.to_excel(writer, sheet_name="servico")
        for k, v in composicoes.items():
            if k.replace("_title", "") != "servico":
                tipo = k.replace("_title", "")
                # print(tipo)
                # display(globals()[f"df_{k.replace('_title', '')}"])
                # display(df[df["projeto"] == s])

                pd.merge(
                    _df_servico,
                    globals()[f"df_{tipo}"],
                    # how="outer",
                    on=["servico", "source"],
                    validate="m:m",
                ).to_excel(writer, sheet_name=tipo)
    # globals()[f"global_{k.replace('_title', '')}"df[df["projeto"] == s])

In [32]:
# import pandas as pd
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl import load_workbook

# wb = load_workbook(filename="target/Projetos.xlsx")
# for s in sheets:
#     _df = df[df["projeto"] == s]
#     ws = wb[s]
#     for k, v in composicoes.items():
#         tipo = k.replace("_title", "")
#         for r in dataframe_to_rows(
#             pd.merge(
#                 _df,
#                 globals()[f"df_{tipo}"],
#                 # how="outer",
#                 on="servico",
#                 validate="m:m",
#             ),
#             index=False,
#             header=True,
#         ):  # No index and don't append the column headers
#             ws.append(r)
#         wb.save("target/Projetos.xlsx")